In [123]:
%matplotlib inline

import math

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

import statsmodels.formula.api as smf
import statsmodels

import seaborn as sn

In [2]:
# read in Inside Airbnb dataframe
df_2020 = pd.read_csv('2020IA.csv',low_memory=False)
df_2022 = pd.read_csv('2022IA.csv',low_memory=False)

In [3]:
# check some example numbers
print(df_2020.head(3))
print(df_2022.head(3))

        id                         listing_url       scrape_id last_scraped  \
0  11551.0  https://www.airbnb.com/rooms/11551  20200824024753   2020-08-26   
1  13913.0  https://www.airbnb.com/rooms/13913  20200824024753   2020-08-27   
2  15400.0  https://www.airbnb.com/rooms/15400  20200824024753   2020-08-26   

                                         name  \
0  Arty and Bright London Apartment in Zone 2   
1         Holiday London DB Room Let-on going   
2         Bright Chelsea  Apartment. Chelsea!   

                                         description  \
0  Unlike most rental apartments out there my fla...   
1  My bright double bedroom with a large window h...   
2  Lots of windows and light.  St Luke's Gardens ...   

                               neighborhood_overview  \
0  Not even 10 minutes by metro from Victoria Sta...   
1  Finsbury Park is a friendly melting pot commun...   
2                                     It is Chelsea.   

                                    

In [4]:
# print the shape of the DataFrame
print(df_2020.shape)
print(df_2022.shape)

(74188, 74)
(66640, 18)


In [5]:
# print the column names
print(df_2020.columns)
print(df_2022.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [6]:
# pick the columns that we need
df_2020_pick = df_2020.loc[:,['id', 'price', 'number_of_reviews_ltm','reviews_per_month']]
df_2022_pick = df_2022.loc[:,['id', 'room_type', 'price','longitude', 'latitude', 'minimum_nights', 'reviews_per_month']]

In [7]:
# rename the columns
df_2020_rename = df_2020_pick.rename(columns = {'price':'price_2020',
                                                'number_of_reviews_ltm':'number_of_reviews_ltm_2020',
                                                'reviews_per_month':'reviews_per_month_2020'})
df_2022_rename = df_2022_pick.rename(columns = {'price':'price_2022',
                                                'reviews_per_month':'reviews_per_month_2022'})
print(df_2020_rename.columns)
print(df_2022_rename.columns)

Index(['id', 'price_2020', 'number_of_reviews_ltm_2020',
       'reviews_per_month_2020'],
      dtype='object')
Index(['id', 'room_type', 'price_2022', 'longitude', 'latitude',
       'minimum_nights', 'reviews_per_month_2022'],
      dtype='object')


In [8]:
# join dataframes
df = pd.merge(df_2020_rename, df_2022_rename, how='inner', on=['id'])

In [9]:
# compair different room types
cnt_room_types = df.groupby('room_type')['id'].count()
print(cnt_room_types)

room_type
Entire home/apt    23002
Hotel room           252
Private room       19790
Shared room          353
Name: id, dtype: int64


In [10]:
#delete $ and change datatype
df['price_2020'] = df['price_2020'].mask(df['price_2020'].str.contains('\,')==True,df['price_2020'].str.replace(',',''))
df['price_2020'] = df['price_2020'].str.split('$').str[1]
df['price_2020'] = df['price_2020'].astype('float64')
#df['number_of_reviews_ltm_2020'] = df['number_of_reviews_ltm_2020'].astype('float64')
#df['reviews_per_month_2020'] = df['reviews_per_month_2020'].astype('float64')
df.dtypes

id                            float64
price_2020                    float64
number_of_reviews_ltm_2020    float64
reviews_per_month_2020        float64
room_type                      object
price_2022                      int64
longitude                     float64
latitude                      float64
minimum_nights                  int64
reviews_per_month_2022        float64
dtype: object

In [11]:
# data cleaning
df['reviews_per_month_2020'] = np.nan_to_num(df['reviews_per_month_2020'])
df['reviews_per_month_2022'] = np.nan_to_num(df['reviews_per_month_2022'])
df_cleaned = df[
    (df["number_of_reviews_ltm_2020"] != 0) & 
    (df["reviews_per_month_2020"] != 0) & 
    (df["reviews_per_month_2022"] != 0) & 
    (df["minimum_nights"] <= 5)]

In [12]:
# create new columns
df_cleaned.loc[:,'price_increase']=(df_cleaned.price_2022 - df_cleaned.price_2020)/df_cleaned.price_2020
df_cleaned.loc[:,'reviews_per_month_increase']=(df_cleaned.reviews_per_month_2022 - df_cleaned.reviews_per_month_2020)/df_cleaned.reviews_per_month_2020
df_cleaned.head()

/tmp/ipykernel_16479/1066492890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.loc[:,'price_increase']=(df_cleaned.price_2022 - df_cleaned.price_2020)/df_cleaned.price_2020
/tmp/ipykernel_16479/1066492890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.loc[:,'reviews_per_month_increase']=(df_cleaned.reviews_per_month_2022 - df_cleaned.reviews_per_month_2020)/df_cleaned.reviews_per_month_2020


,id,price_2020,number_of_reviews_ltm_2020,reviews_per_month_2020,room_type,price_2022,longitude,latitude,minimum_nights,reviews_per_month_2022,price_increase,reviews_per_month_increase
0,15400.0,75.0,3.0,0.68,Entire home/apt,75,-0.16813,51.48780,3,0.60,0.000000,-0.117647
1,17402.0,208.0,1.0,0.37,Entire home/apt,307,-0.14094,51.52195,4,0.34,0.475962,-0.081081
4,33332.0,65.0,1.0,0.05,Private room,65,-0.32554,51.46416,2,0.09,0.000000,0.800000
5,36299.0,195.0,4.0,0.64,Entire home/apt,195,-0.28086,51.48085,3,0.64,0.000000,0.000000
6,36660.0,72.0,19.0,4.36,Private room,72,-0.16057,51.58478,2,3.93,0.000000,-0.098624


In [13]:
# data cleaning again
df_cleaned2 = df_cleaned[(df_cleaned["price_increase"] != 0)]

In [14]:
df_pick = df_cleaned2.loc[:,['longitude', 'latitude', 'price_increase', 'reviews_per_month_increase']]
df_pick=df_pick.reset_index(drop=True)
print(df_pick.head())
print(df_pick.dtypes)

   longitude  latitude  price_increase  reviews_per_month_increase
0   -0.14094  51.52195        0.475962                   -0.081081
1   -0.23345  51.50681        0.068966                   -0.127451
2   -0.25632  51.61492        2.121099                   -0.190476
3   -0.25568  51.49351        2.571429                   -0.142857
4   -0.10204  51.50191        0.144928                   -0.177215
longitude                     float64
latitude                      float64
price_increase                float64
reviews_per_month_increase    float64
dtype: object


In [15]:
# read in London sites dataframe
df_sites = pd.read_csv('vva_site_listings_London_2021_02.csv')

In [16]:
# print the column names
print(df_sites.columns)

Index(['Provider_Product_ID', 'Attraction', 'Region', 'County  ', 'District',
       'CRU', 'Category', 'Visitors 2016', 'Visitors 2017', 'Visitors 2018',
       'Visitors 2019', 'Visitors 2020', 'Visitors 2021', 'Estimate?',
       'Visitor Admission bracket', '% change in admissions 2020 to 2021',
       '% change in admissions 2019 to 2021', 'Charging', 'Charge band',
       'Longitude', 'Latitude'],
      dtype='object')


In [17]:
# pick the columns that we need
df_sites_pick = df_sites.loc[:,['Longitude','Latitude', 'Visitors 2020','Visitors 2021']]
print(df_sites_pick.head())
print(df_sites_pick.dtypes)

   Longitude   Latitude Visitors 2020 Visitors 2021
0  -0.295909  51.479362       1222956    1,963,171 
1  -0.176389  51.496111       1296763    1,571,413 
2  -0.126944  51.519444       1275466    1,327,120 
3  -0.099400  51.507600       1431704    1,156,037 
4  -0.117194  51.511028        724310      984,978 
Longitude        float64
Latitude         float64
Visitors 2020     object
Visitors 2021     object
dtype: object


In [30]:
#Dealing with Geo Data of sites

gdf_houses = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_pick.longitude, df_pick.latitude, crs='epsg:4326'))
gdf_houses = gdf_houses.geometry.to_crs('epsg:27700')
print(gdf_houses.head())

gdf_sites = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_sites_pick.Longitude, df_sites_pick.Latitude, crs='epsg:4326'))
gdf_sites = gdf_sites.geometry.to_crs('epsg:27700')
print(gdf_sites.head())


0    POINT (529075.790 181974.865)
1    POINT (522698.676 180132.232)
2    POINT (520824.939 192116.667)
3    POINT (521191.223 178616.205)
4    POINT (531832.196 179815.633)
Name: geometry, dtype: geometry
0    POINT (518435.241 176977.100)
1    POINT (526688.169 179039.579)
2    POINT (530053.870 181720.949)
3    POINT (531998.990 180453.132)
4    POINT (530754.373 180802.421)
Name: geometry, dtype: geometry


In [ ]:
def changeencode(data, cols):
    for col in cols:
        data[col] = data[col].str.decode('iso-8859-1').str.encode('utf-8')
    return data   

In [231]:
#for i in range(0,len(gdf_houses)):
#def houses_to_sites(i,j):
for i in range(0,50):
    #(x1,y1) is the coordinate of houses
    x1=gdf_houses.x[i]
    y1=gdf_houses.y[i]
    r=1000 #radius 1km
    count=0
    for j in range(0,len(gdf_sites)):
        #(x2,y2) is the coordinate of sites
        x2=gdf_sites.x[j]
        y2=gdf_sites.y[j]
        d=pow(pow(x1-x2,2)+pow(y1-y2,2),0.5)
        if d<r:
            count+=1
            houses_to_sites=[i, j, count, d]
            #k=0
            #for house_to_site in houses_to_sites:
            #    print("{0}, {1}, {2}, {3}".format(str(houses_to_sites[0]), str(houses_to_sites[1]), str(houses_to_sites[2]), str(houses_to_sites[3])))
            #    k = k+1
            ##to excel
            #houses_to_sites=[i,j,count,d]
            #df_houses_to_sites=pd.DataFrame([houses_to_sites],columns = ['house_index', 'site_index', 'count', 'd'])
            #with pd.ExcelWriter('test.xlsx') as writer:
            #    df_houses_to_sites.to_excel(writer, encoding='utf8', sheet_name='data')
        #df_houses_to_sites.to_csv ("test.csv",index=0)
                #data = pd.DataFrame({'i': i, 'j': j}, index=[0])
            #return(df_houses_to_sites)
            
            #dict={'count':[count], 'x1':[x1], 'y1':[y1],'x2':[x2], 'y1':[y2], 'd':[d]}
            #test=test.append(pd.DataFrame({'count':[count], 'x1':[x1], 'y1':[y1],'x2':[x2], 'y1':[y2], 'd':[d]}))
            #test=test.append(pd.DataFrame(count,x1,y1,x2,y2,d),ignore_index=True)

            #df1 = pd.DataFrame(count,x1,y1,x2,y2,d)
            #print(count, x1, y1, x2, y2, d)
            #houses_to_sites_tuple=list(zip(count, x1, y1, x2, y2, d))
#                houses_to_sites=i,j,count,x1,y1,x2,y2,d
#    return houses_to_sites_df = pd.DataFrame(houses_to_sites,axis=0)
            #test = pd.DataFrame(houses_to_sites_tuple,columns = ['count', 'x1', 'y1', 'x2', 'y2', 'd'])

0, 41, 1, 517.875515086801
0, 41, 1, 517.875515086801
0, 41, 1, 517.875515086801
0, 41, 1, 517.875515086801
0, 51, 2, 434.72255303552487
0, 51, 2, 434.72255303552487
0, 51, 2, 434.72255303552487
0, 51, 2, 434.72255303552487
3, 46, 1, 806.8507719670265
3, 46, 1, 806.8507719670265
3, 46, 1, 806.8507719670265
3, 46, 1, 806.8507719670265
4, 3, 1, 658.957940551293
4, 3, 1, 658.957940551293
4, 3, 1, 658.957940551293
4, 3, 1, 658.957940551293
4, 15, 2, 817.7271626094182
4, 15, 2, 817.7271626094182
4, 15, 2, 817.7271626094182
4, 15, 2, 817.7271626094182
4, 17, 3, 761.1158517521242
4, 17, 3, 761.1158517521242
4, 17, 3, 761.1158517521242
4, 17, 3, 761.1158517521242
17, 2, 1, 885.7306007105306
17, 2, 1, 885.7306007105306
17, 2, 1, 885.7306007105306
17, 2, 1, 885.7306007105306
17, 41, 2, 424.84349400962185
17, 41, 2, 424.84349400962185
17, 41, 2, 424.84349400962185
17, 41, 2, 424.84349400962185
17, 51, 3, 306.5679627901606
17, 51, 3, 306.5679627901606
17, 51, 3, 306.5679627901606
17, 51, 3, 306.56

In [56]:
df_houses_to_sites = pd.DataFrame(houses_to_sites, columns = ['count', 'x1', 'y1', 'x2', 'y2', 'd'])
print(df_houses_to_sites.head())

ValueError: Shape of passed values is (6, 1), indices imply (6, 6)

In [48]:
#            houses_to_sites = pd.DataFrame(list(zip(count,x1,y1,x2,y2,d)),
#                                           columns=['count', 'x1', 'y1', 'x2', 'y2', 'd'])
# convert the list into dataframe row
houses_to_sites.to_excel
data = pd.DataFrame(houses_to_sites).T
 
# add columns
data.columns = ['count', 'x1', 'y1','x2', 'y2', 'd']
data1 = houses_to_sites.groupby('count').apply(lambda t: t[(t['d']==t['d'].min()) ^ (t['grade']==t['grade'].max())])

AttributeError: 'list' object has no attribute 'groupby'